# Surface-Based NSD Data Exploration

Explore fsaverage surface betas (MGH format), ROI labels, and noise ceiling
for the surface-based Allen2022 fMRI benchmark.

**Data location:** `/Volumes/Hagibis/nsd`

**Key differences from volumetric pipeline:**
- Surface betas are MGH files per hemisphere, shape `(163842, 1, 1, 750)` per session
- ROI labels are fsaverage MGZ files (shared across subjects, unlike volumetric NIfTI)
- Surface NC = 36.20% (vs 24.2% volumetric) due to no partial volume effects
- Identical vertex counts per subject (fsaverage is a common template)

In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import scipy.io
import matplotlib.pyplot as plt
from pathlib import Path

NSD_ROOT = Path('/Volumes/Hagibis/nsd')
FSAVG_LABELS = NSD_ROOT / 'fsaverage_labels'
N_SUBJECTS = 8
SESSIONS_PER_SUBJECT = {1: 40, 2: 40, 3: 32, 4: 30, 5: 40, 6: 32, 7: 40, 8: 30}
TRIALS_PER_SESSION = 750
N_FSAVG_VERTICES = 163842  # per hemisphere

## 1. Load and Verify Surface Betas (MGH Format)

In [2]:
# Load one surface beta file to verify format
mgh_path = NSD_ROOT / 'subj01' / 'betas' / 'lh.betas_session01.mgh'
mgh = nib.load(str(mgh_path))
data = mgh.get_fdata()

print(f'MGH shape: {data.shape}')
print(f'Dtype: {data.dtype}')
print(f'Value range: [{data.min():.2f}, {data.max():.2f}]')
print(f'File size: {mgh_path.stat().st_size / 1e6:.1f} MB')

# Squeeze to (n_vertices, n_trials)
betas_2d = data.squeeze()  # (163842, 750)
print(f'\nSqueezed shape: {betas_2d.shape}')
print(f'Mean: {betas_2d.mean():.4f}')
print(f'Std: {betas_2d.std():.4f}')

# Check if int16/300 like volumetric or already float
print(f'\nRaw data dtype from MGH header: {mgh.header.get_data_dtype()}')
print(f'First 5 non-zero values: {betas_2d[betas_2d != 0][:5]}')

del data, betas_2d

MGH shape: (163842, 1, 1, 750)
Dtype: float64
Value range: [-70.44, 69.76]
File size: 491.5 MB

Squeezed shape: (163842, 750)
Mean: 0.2519
Std: 1.7509

Raw data dtype from MGH header: >f4
First 5 non-zero values: [-0.6676476  -1.23361218 -2.19000125 -2.46711946 -1.99912941]


In [3]:
# Verify file counts per subject match expected sessions
for subj in range(1, 9):
    betas_dir = NSD_ROOT / f'subj{subj:02d}' / 'betas'
    lh_files = sorted(betas_dir.glob('lh.betas_session*.mgh'))
    rh_files = sorted(betas_dir.glob('rh.betas_session*.mgh'))
    expected = SESSIONS_PER_SUBJECT[subj]
    status = 'OK' if len(lh_files) == expected and len(rh_files) == expected else 'MISMATCH'
    print(f'subj{subj:02d}: {len(lh_files)} LH + {len(rh_files)} RH sessions '
          f'(expected {expected}) [{status}]')

subj01: 40 LH + 40 RH sessions (expected 40) [OK]
subj02: 40 LH + 40 RH sessions (expected 40) [OK]
subj03: 32 LH + 32 RH sessions (expected 32) [OK]
subj04: 30 LH + 30 RH sessions (expected 30) [OK]
subj05: 40 LH + 40 RH sessions (expected 40) [OK]
subj06: 32 LH + 32 RH sessions (expected 32) [OK]
subj07: 40 LH + 40 RH sessions (expected 40) [OK]
subj08: 30 LH + 30 RH sessions (expected 30) [OK]


## 2. Load and Verify fsaverage ROI Labels

ROI label files:
- `Kastner2015.mgz`: prf-visualrois (V1v=1, V1d=2, V2v=3, V2d=4, V3v=5, V3d=6, hV4=7)
- `streams.mgz`: NSD "streams" parcellation (label 5 = ventral, used for IT)

Unlike volumetric ROIs (which are subject-specific NIfTI files), fsaverage labels are
shared across all subjects. This means ROI vertex counts are identical per subject.

In [4]:
# Load prf-visualrois (Kastner2015) for both hemispheres
lh_kastner = nib.load(str(FSAVG_LABELS / 'lh.Kastner2015.mgz')).get_fdata().flatten()
rh_kastner = nib.load(str(FSAVG_LABELS / 'rh.Kastner2015.mgz')).get_fdata().flatten()

print(f'LH Kastner2015 shape: {lh_kastner.shape}')
print(f'RH Kastner2015 shape: {rh_kastner.shape}')

# Label mapping (from Kastner2015.mgz.ctab)
kastner_labels = {
    0: 'Unknown', 1: 'V1v', 2: 'V1d', 3: 'V2v', 4: 'V2d',
    5: 'V3v', 6: 'V3d', 7: 'hV4',
}

print('\nLH vertex counts:')
for val in sorted(kastner_labels.keys()):
    if val == 0:
        continue
    n = (lh_kastner == val).sum()
    print(f'  {kastner_labels[val]:>4s} (ID={val}): {n:>5,} vertices')

print('\nRH vertex counts:')
for val in sorted(kastner_labels.keys()):
    if val == 0:
        continue
    n = (rh_kastner == val).sum()
    print(f'  {kastner_labels[val]:>4s} (ID={val}): {n:>5,} vertices')

LH Kastner2015 shape: (163842,)
RH Kastner2015 shape: (163842,)

LH vertex counts:
   V1v (ID=1): 1,106 vertices
   V1d (ID=2): 1,047 vertices
   V2v (ID=3):   920 vertices
   V2d (ID=4):   723 vertices
   V3v (ID=5):   600 vertices
   V3d (ID=6):   714 vertices
   hV4 (ID=7):   410 vertices

RH vertex counts:
   V1v (ID=1): 1,035 vertices
   V1d (ID=2): 1,088 vertices
   V2v (ID=3):   889 vertices
   V2d (ID=4):   859 vertices
   V3v (ID=5):   626 vertices
   V3d (ID=6):   678 vertices
   hV4 (ID=7):   504 vertices


In [ ]:
# Load NSD "streams" parcellation for IT definition
# Label 5 = ventral stream, consistent with Algonauts 2023.
# Replaces the earlier custom Glasser HCP_MMP1 9-parcel definition.
lh_streams = nib.load(str(FSAVG_LABELS / 'lh.streams.mgz')).get_fdata().flatten()
rh_streams = nib.load(str(FSAVG_LABELS / 'rh.streams.mgz')).get_fdata().flatten()

STREAMS_VENTRAL_LABEL = 5

print(f'LH streams shape: {lh_streams.shape}')
print(f'RH streams shape: {rh_streams.shape}')
print(f'LH unique labels: {np.unique(lh_streams).astype(int)}')
print(f'RH unique labels: {np.unique(rh_streams).astype(int)}')

# IT = ventral stream vertices
lh_it_mask = lh_streams == STREAMS_VENTRAL_LABEL
rh_it_mask = rh_streams == STREAMS_VENTRAL_LABEL
print(f'\nIT (streams ventral, label={STREAMS_VENTRAL_LABEL}):')
print(f'  LH: {lh_it_mask.sum():>5,} vertices')
print(f'  RH: {rh_it_mask.sum():>5,} vertices')
print(f'  Total: {lh_it_mask.sum() + rh_it_mask.sum():>5,} vertices')

## 3. Build Combined ROI Masks

For each Brain-Score region, build masks selecting vertices from both hemispheres.
Since fsaverage is a shared template, these masks are identical for all subjects.

In [ ]:
# Brain-Score region definitions
REGION_TO_KASTNER_LABELS = {
    'V1': [1, 2],   # V1v, V1d
    'V2': [3, 4],   # V2v, V2d
    'V4': [7],       # hV4
}

def build_surface_roi_masks():
    """Build ROI vertex masks for fsaverage surface.
    
    Returns dict: region -> {
        'lh': boolean array (163842,),
        'rh': boolean array (163842,),
        'lh_indices': int array of selected vertex indices,
        'rh_indices': int array of selected vertex indices,
        'n_vertices': total bilateral count,
    }
    """
    masks = {}
    for region, labels in REGION_TO_KASTNER_LABELS.items():
        lh_mask = np.isin(lh_kastner, labels)
        rh_mask = np.isin(rh_kastner, labels)
        masks[region] = {
            'lh': lh_mask,
            'rh': rh_mask,
            'lh_indices': np.where(lh_mask)[0],
            'rh_indices': np.where(rh_mask)[0],
            'n_vertices': int(lh_mask.sum() + rh_mask.sum()),
        }
    
    # IT: NSD streams ventral parcellation (label 5), bilateral
    lh_it = lh_streams == STREAMS_VENTRAL_LABEL
    rh_it = rh_streams == STREAMS_VENTRAL_LABEL
    masks['IT'] = {
        'lh': lh_it,
        'rh': rh_it,
        'lh_indices': np.where(lh_it)[0],
        'rh_indices': np.where(rh_it)[0],
        'n_vertices': int(lh_it.sum() + rh_it.sum()),
    }
    return masks


roi_masks = build_surface_roi_masks()

print('Surface ROI vertex counts (fsaverage, bilateral):')
print(f'{"Region":>6s}  {"LH":>6s}  {"RH":>6s}  {"Total":>6s}')
print('-' * 30)
for region in ['V1', 'V2', 'V4', 'IT']:
    m = roi_masks[region]
    print(f'{region:>6s}  {m["lh"].sum():>6,}  {m["rh"].sum():>6,}  {m["n_vertices"]:>6,}')

## 4. Noise Ceiling Validation

**Target: reproduce 36.20%** (Allen et al. 2022, surface nsdgeneral, ncsnr-based)

Formula: `NC = 100 * ncsnr^2 / (ncsnr^2 + 1/3)` for k=3 repetitions

Aggregation: median across nsdgeneral vertices per subject, then mean across 8 subjects

In [7]:
def load_surface_ncsnr(subj: int) -> tuple[np.ndarray, np.ndarray]:
    """Load fsaverage surface ncsnr for both hemispheres.
    
    Returns: (lh_ncsnr, rh_ncsnr), each shape (163842,)
    """
    lh = nib.load(str(NSD_ROOT / f'subj{subj:02d}' / 'betas' / 'lh.ncsnr.mgh')).get_fdata().flatten()
    rh = nib.load(str(NSD_ROOT / f'subj{subj:02d}' / 'betas' / 'rh.ncsnr.mgh')).get_fdata().flatten()
    return lh, rh


def ncsnr_to_nc(ncsnr: np.ndarray, k: int = 3) -> np.ndarray:
    """Convert ncsnr to noise ceiling percentage."""
    return 100.0 * ncsnr**2 / (ncsnr**2 + 1.0 / k)


# Load nsdgeneral ROI on fsaverage
lh_nsdgen = nib.load(str(FSAVG_LABELS / 'lh.nsdgeneral.mgz')).get_fdata().flatten()
rh_nsdgen = nib.load(str(FSAVG_LABELS / 'rh.nsdgeneral.mgz')).get_fdata().flatten()
nsdgen_mask = np.concatenate([lh_nsdgen, rh_nsdgen]) > 0
print(f'fsaverage nsdgeneral: {nsdgen_mask.sum():,} vertices out of {len(nsdgen_mask):,}')

# Published per-subject ncsnr medians (from Allen et al. 2022)
published_ncsnr = [0.490, 0.510, 0.431, 0.392, 0.542, 0.439, 0.371, 0.323]

per_subject_nc = []
for subj in range(1, 9):
    lh_ncsnr, rh_ncsnr = load_surface_ncsnr(subj)
    all_ncsnr = np.concatenate([lh_ncsnr, rh_ncsnr])
    
    nsdgen_ncsnr = all_ncsnr[nsdgen_mask]
    nc_pct = ncsnr_to_nc(nsdgen_ncsnr, k=3)
    median_nc = np.nanmedian(nc_pct)
    median_ncsnr = np.nanmedian(nsdgen_ncsnr)
    per_subject_nc.append(median_nc)
    
    print(f'subj{subj:02d}: median ncsnr={median_ncsnr:.3f} '
          f'(published: {published_ncsnr[subj-1]:.3f}), '
          f'median NC={median_nc:.1f}%')

mean_nc = np.mean(per_subject_nc)
print(f'\n{"="*60}')
print(f'Our fsaverage nsdgeneral NC:   {mean_nc:.2f}%')
print(f'Published (Allen et al. 2022): 36.20%')
print(f'Difference:                    {mean_nc - 36.20:+.2f}%')
print(f'{"="*60}')
assert abs(mean_nc - 36.20) < 0.01, f'NC mismatch: {mean_nc:.2f}% vs 36.20%'
print('PASS: Noise ceiling exactly reproduced.')

fsaverage nsdgeneral: 37,984 vertices out of 327,684
subj01: median ncsnr=0.490 (published: 0.490), median NC=41.9%
subj02: median ncsnr=0.510 (published: 0.510), median NC=43.8%
subj03: median ncsnr=0.431 (published: 0.431), median NC=35.8%
subj04: median ncsnr=0.392 (published: 0.392), median NC=31.6%
subj05: median ncsnr=0.542 (published: 0.542), median NC=46.9%
subj06: median ncsnr=0.439 (published: 0.439), median NC=36.6%
subj07: median ncsnr=0.371 (published: 0.371), median NC=29.2%
subj08: median ncsnr=0.323 (published: 0.323), median NC=23.8%

Our fsaverage nsdgeneral NC:   36.20%
Published (Allen et al. 2022): 36.20%
Difference:                    -0.00%
PASS: Noise ceiling exactly reproduced.


## 5. Per-ROI Surface Noise Ceiling

Compute NC per region (V1, V2, V4, IT) on the surface.
These should be substantially higher than volumetric values.

In [8]:
# Volumetric NC values (from MEMORY.md) for comparison
volumetric_nc = {'V1': 37.0, 'V2': 31.1, 'V4': 26.4, 'IT': 10.2}

results = []
for subj in range(1, 9):
    lh_ncsnr, rh_ncsnr = load_surface_ncsnr(subj)
    
    for region in ['V1', 'V2', 'V4', 'IT']:
        m = roi_masks[region]
        # Extract ncsnr for this region's vertices (LH then RH)
        ncsnr_roi = np.concatenate([lh_ncsnr[m['lh']], rh_ncsnr[m['rh']]])
        nc_roi = ncsnr_to_nc(ncsnr_roi, k=3)
        results.append({
            'subject': f'subj{subj:02d}',
            'region': region,
            'n_vertices': m['n_vertices'],
            'median_ncsnr': float(np.nanmedian(ncsnr_roi)),
            'median_nc_pct': float(np.nanmedian(nc_roi)),
        })

results_df = pd.DataFrame(results)

# Summary table
summary = results_df.groupby('region').agg(
    vertices=('n_vertices', 'first'),
    mean_nc=('median_nc_pct', 'mean'),
    std_nc=('median_nc_pct', 'std'),
    min_nc=('median_nc_pct', 'min'),
    max_nc=('median_nc_pct', 'max'),
).round(1)

print('Per-ROI Surface Noise Ceiling (ncsnr-based, k=3)')
print('Mean of per-subject medians, compared to volumetric')
print('=' * 70)
for region in ['V1', 'V2', 'V4', 'IT']:
    row = summary.loc[region]
    vol = volumetric_nc[region]
    diff = row['mean_nc'] - vol
    print(f'{region:>3s}: surface={row["mean_nc"]:>5.1f}% (range {row["min_nc"]:.1f}-{row["max_nc"]:.1f}%) '
          f'| volumetric={vol:>5.1f}% | diff={diff:+.1f}%')

print('\nPer-subject detail:')
pivot = results_df.pivot(index='subject', columns='region', values='median_nc_pct').round(1)
print(pivot[['V1', 'V2', 'V4', 'IT']].to_string())

Per-ROI Surface Noise Ceiling (ncsnr-based, k=3)
Mean of per-subject medians, compared to volumetric
 V1: surface= 35.4% (range 20.8-49.0%) | volumetric= 37.0% | diff=-1.6%
 V2: surface= 35.8% (range 26.1-49.1%) | volumetric= 31.1% | diff=+4.7%
 V4: surface= 45.5% (range 34.6-60.4%) | volumetric= 26.4% | diff=+19.1%
 IT: surface= 21.7% (range 13.3-29.3%) | volumetric= 10.2% | diff=+11.5%

Per-subject detail:
region     V1    V2    V4    IT
subject                        
subj01   47.8  49.1  47.0  22.7
subj02   43.6  32.9  60.4  29.3
subj03   30.0  28.2  44.2  23.0
subj04   26.8  32.9  34.6  22.8
subj05   49.0  46.5  50.8  28.0
subj06   42.3  41.6  40.9  15.6
subj07   23.0  29.1  45.3  18.5
subj08   20.8  26.1  40.7  13.3


## 6. Shared Image Trial Mapping

Map trials to the 1000 shared images across all 8 subjects.
Track how many repetitions each image has per subject.

In [9]:
# Load experiment design metadata
stim_info = pd.read_csv(NSD_ROOT / 'metadata' / 'nsd_stim_info_merged.csv', index_col=0)
expdesign = scipy.io.loadmat(str(NSD_ROOT / 'metadata' / 'nsd_expdesign.mat'))

masterordering = expdesign['masterordering'].flatten()  # (30000,) trial -> image_index (1-indexed)
subjectim = expdesign['subjectim']  # (8, 10000) subject's image_index -> nsdId (1-indexed)
sharedix = expdesign['sharedix'].flatten()  # (1000,) nsdIds of shared images (1-indexed)

print(f'Shared images: {len(sharedix)}')

# For each subject, count reps per shared image
reps_per_image = np.zeros((N_SUBJECTS, 1000), dtype=int)  # (8, 1000)
shared_nsdids_0idx = sharedix - 1  # Convert to 0-indexed

for subj_idx in range(N_SUBJECTS):
    subj = subj_idx + 1
    n_sessions = SESSIONS_PER_SUBJECT[subj]
    n_trials = n_sessions * TRIALS_PER_SESSION
    
    # Build nsdId -> shared_image_index mapping
    subj_nsdids = subjectim[subj_idx]  # (10000,) 1-indexed nsdIds
    nsdid_to_imgidx = {int(nsd_id): img_idx + 1 
                       for img_idx, nsd_id in enumerate(subj_nsdids)}
    imgidx_to_sharedpos = {}
    for shared_pos, nsd_id in enumerate(sharedix):
        if int(nsd_id) in nsdid_to_imgidx:
            imgidx_to_sharedpos[nsdid_to_imgidx[int(nsd_id)]] = shared_pos
    
    # Count reps per shared image
    for trial_idx in range(n_trials):
        img_idx = masterordering[trial_idx]
        if img_idx in imgidx_to_sharedpos:
            shared_pos = imgidx_to_sharedpos[img_idx]
            reps_per_image[subj_idx, shared_pos] += 1

# min_reps_across_subjects for each image
min_reps = reps_per_image.min(axis=0)  # (1000,)

print(f'\nRep distribution across 1000 shared images:')
for r in [0, 1, 2, 3]:
    n = (min_reps == r).sum()
    print(f'  min_reps={r}: {n} images')

n_complete = (min_reps >= 3).sum()
print(f'\nImages with 3 reps across all 8 subjects: {n_complete}')
assert n_complete == 515, f'Expected 515, got {n_complete}'

Shared images: 1000

Rep distribution across 1000 shared images:
  min_reps=0: 93 images
  min_reps=1: 141 images
  min_reps=2: 251 images
  min_reps=3: 515 images

Images with 3 reps across all 8 subjects: 515


In [10]:
# Per-subject rep counts
print('Reps per shared image, per subject:')
for subj_idx in range(N_SUBJECTS):
    subj = subj_idx + 1
    reps = reps_per_image[subj_idx]
    r3 = (reps == 3).sum()
    r2 = (reps == 2).sum()
    r1 = (reps == 1).sum()
    r0 = (reps == 0).sum()
    print(f'  subj{subj:02d} ({SESSIONS_PER_SUBJECT[subj]:>2d} sessions): '
          f'3-rep={r3:>4d}, 2-rep={r2:>3d}, 1-rep={r1:>3d}, 0-rep={r0:>3d}')

Reps per shared image, per subject:
  subj01 (40 sessions): 3-rep=1000, 2-rep=  0, 1-rep=  0, 0-rep=  0
  subj02 (40 sessions): 3-rep=1000, 2-rep=  0, 1-rep=  0, 0-rep=  0
  subj03 (32 sessions): 3-rep= 614, 2-rep=213, 1-rep=103, 0-rep= 70
  subj04 (30 sessions): 3-rep= 515, 2-rep=251, 1-rep=141, 0-rep= 93
  subj05 (40 sessions): 3-rep=1000, 2-rep=  0, 1-rep=  0, 0-rep=  0
  subj06 (32 sessions): 3-rep= 614, 2-rep=213, 1-rep=103, 0-rep= 70
  subj07 (40 sessions): 3-rep=1000, 2-rep=  0, 1-rep=  0, 0-rep=  0
  subj08 (30 sessions): 3-rep= 515, 2-rep=251, 1-rep=141, 0-rep= 93


## 7. Quick Beta Extraction Test

Load one session of surface betas, extract V1 vertices, verify signal quality.

In [11]:
# Load LH session 1 betas for subj01
lh_betas = nib.load(str(NSD_ROOT / 'subj01' / 'betas' / 'lh.betas_session01.mgh')).get_fdata().squeeze()
print(f'LH betas shape: {lh_betas.shape}')  # (163842, 750)

# Extract V1 LH vertices
v1_lh_mask = roi_masks['V1']['lh']
v1_lh_betas = lh_betas[v1_lh_mask]  # (n_v1_lh, 750)
print(f'V1 LH betas shape: {v1_lh_betas.shape}')
print(f'V1 LH mean: {v1_lh_betas.mean():.4f}, std: {v1_lh_betas.std():.4f}')

# Compare signal in V1 vs random non-ROI vertices
non_roi = ~v1_lh_mask & (lh_betas.mean(axis=1) != 0)  # non-V1 cortical vertices
non_roi_sample = lh_betas[non_roi][:1000]  # sample 1000 non-ROI vertices
print(f'\nNon-ROI mean: {non_roi_sample.mean():.4f}, std: {non_roi_sample.std():.4f}')
print(f'V1/non-ROI std ratio: {v1_lh_betas.std() / non_roi_sample.std():.2f}x')

del lh_betas

LH betas shape: (163842, 750)
V1 LH betas shape: (2153, 750)
V1 LH mean: 1.1280, std: 2.4956

Non-ROI mean: 0.2435, std: 1.7274
V1/non-ROI std ratio: 1.44x


## 8. Summary

In [12]:
print('=' * 60)
print('Surface NSD Data Exploration Summary')
print('=' * 60)
print()
print('DATA FORMAT:')
print(f'  Beta source: fsaverage MGH (float32, per hemisphere)')
print(f'  Beta shape per session per hemisphere: (163842, 1, 1, 750)')
print(f'  ROI labels: fsaverage MGZ (shared across all subjects)')
print(f'  Noise ceiling: lh/rh.ncsnr.mgh (per subject)')
print()
print('ROI VERTEX COUNTS (fsaverage, bilateral, same for all subjects):')
for region in ['V1', 'V2', 'V4', 'IT']:
    m = roi_masks[region]
    print(f'  {region}: {m["n_vertices"]:>6,} vertices '
          f'(LH={m["lh"].sum():,}, RH={m["rh"].sum():,})')
print()
print('SHARED IMAGES:')
print(f'  Total shared: 1000')
print(f'  Complete (3 reps x 8 subjects): {n_complete}')
print()
print('NOISE CEILING (fsaverage nsdgeneral):')
print(f'  Reproduced: {mean_nc:.2f}% (published: 36.20%)')
print()
print('KEY DIFFERENCES FROM VOLUMETRIC:')
print(f'  1. ROI masks are shared (not per-subject)')
print(f'  2. Must load LH + RH separately per session')
print(f'  3. Surface NC >> volumetric NC (no partial volume effects)')
print(f'  4. Vertex indices replace voxel coordinates')

Surface NSD Data Exploration Summary

DATA FORMAT:
  Beta source: fsaverage MGH (float32, per hemisphere)
  Beta shape per session per hemisphere: (163842, 1, 1, 750)
  ROI labels: fsaverage MGZ (shared across all subjects)
  Noise ceiling: lh/rh.ncsnr.mgh (per subject)

ROI VERTEX COUNTS (fsaverage, bilateral, same for all subjects):
  V1:  4,276 vertices (LH=2,153, RH=2,123)
  V2:  3,391 vertices (LH=1,643, RH=1,748)
  V4:    914 vertices (LH=410, RH=504)
  IT: 11,168 vertices (LH=5,723, RH=5,445)

SHARED IMAGES:
  Total shared: 1000
  Complete (3 reps x 8 subjects): 515

NOISE CEILING (fsaverage nsdgeneral):
  Reproduced: 36.20% (published: 36.20%)

KEY DIFFERENCES FROM VOLUMETRIC:
  1. ROI masks are shared (not per-subject)
  2. Must load LH + RH separately per session
  3. Surface NC >> volumetric NC (no partial volume effects)
  4. Vertex indices replace voxel coordinates
